# LegacyHalos Sample Selection

The goal of this notebook is to select a sample of central and satellite galaxies from redMaPPer but with (deeper) Legacy Survey photometry from DR9.

The input (row-matched) catalogs for the centrals are:
* \$REDMAPPER_DIR/v6.3.1/dr8_run_redmapper_v6.3.1_lgt5_catalog.fit  
* \$REDMAPPER_DIR/v6.3.1/redmapper-v6.3.1-lgt5-centrals-sdssWISEphot-dr14.fits (generated by [redmapper-sdssWISEphot.ipynb](https://github.com/moustakas/legacyhalos/tree/master/doc/redmapper-sdssWISEphot.ipynb))  
* \$REDMAPPER_DIR/v6.3.1/legacysurvey-dr9-north-centrals-v6.3.1.fits (generated by [legacyhalos-match-redmapper](https://github.com/moustakas/legacyhalos/tree/master/bin/legacyhalos/legacyhalos-match-redmapper))  
* \$REDMAPPER_DIR/v6.3.1/legacysurvey-dr9-south-centrals-v6.3.1.fits (generated by [legacyhalos-match-redmapper](https://github.com/moustakas/legacyhalos/tree/master/bin/legacyhalos/legacyhalos-match-redmapper)) 


and for the satellites:
* \$REDMAPPER_DIR/v6.3.1/dr8_run_redmapper_v6.3.1_lgt5_catalog_members.fit
* \$REDMAPPER_DIR/v6.3.1/redmapper-v6.3.1-lgt5-members-sdssWISEphot-dr14.fits (generated by [redmapper-sdssWISEphot.ipynb](https://github.com/moustakas/legacyhalos/tree/master/doc/redmapper-sdssWISEphot.ipynb))  
* \$REDMAPPER_DIR/v6.3.1/legacysurvey-dr9-north-members-v6.3.1.fits (generated by [legacyhalos-match-redmapper](https://github.com/moustakas/legacyhalos/tree/master/bin/legacyhalos/legacyhalos-match-redmapper))
* \$REDMAPPER_DIR/v6.3.1/legacysurvey-dr9-south-members-v6.3.1.fits (generated by [legacyhalos-match-redmapper](https://github.com/moustakas/legacyhalos/tree/master/bin/legacyhalos/legacyhalos-match-redmapper))

And the resulting output catalogs are the files:

* \$LEGACYHALOS_DIR/sample/legacyhalos-centrals-dr9.fits
* \$LEGACYHALOS_DIR/sample/legacyhalos-candidate-centrals-dr9.fits

In addition, we create jackknife subsamples of the data and write them out in the file:

* \$LEGACYHALOS_DIR/sample/legacyhalos-jackknife-dr9.fits

### Paths and Imports

In production, these environment variables are set in a shell script.

In [2]:
import sys
%set_env LEGACYHALOS_DIR=/global/cfs/cdirs/desi/users/ioannis/legacyhalos
%set_env LEGACYHALOS_DATA_DIR=/global/cscratch1/sd/ioannis/legacyhalos-data
%set_env LEGACYHALOS_HTML_DIR=/global/cfs/cdirs/cosmo/www/temp/ioannis/legacyhalos-html
%set_env REDMAPPER_DIR=/global/cfs/cdirs/desi/users/ioannis/redmapper
sys.path.insert(0, '/global/u2/i/ioannis/repos/git/legacyhalos/py')

env: LEGACYHALOS_DIR=/global/cfs/cdirs/desi/users/ioannis/legacyhalos
env: LEGACYHALOS_DATA_DIR=/global/cscratch1/sd/ioannis/legacyhalos-data
env: LEGACYHALOS_HTML_DIR=/global/cfs/cdirs/cosmo/www/temp/ioannis/legacyhalos-html
env: REDMAPPER_DIR=/global/cfs/cdirs/desi/users/ioannis/redmapper


In [3]:
import os, sys, warnings, pdb, time
import numpy as np
import matplotlib.pyplot as plt

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-1pphi8da because the default path (/homedir/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [4]:
import pandas as pd
import healpy as hp
import fitsio
from astropy.io import fits
from astropy.table import Table, Column, vstack, hstack
import astropy.units as u
from astropy.coordinates import SkyCoord

In [5]:
import legacyhalos.io
from legacyhalos.misc import radec2pix, pix2radec

In [6]:
import seaborn as sns
%matplotlib inline

In [7]:
#index = np.arange(50000)
index = None

### Specify the LegacyHalos path and output files names.

In [9]:
lsdr, sdssdr, rmversion = 'dr9', 'dr14', 'v6.3.1'

In [10]:
from legacyhalos.legacyhalos import sample_dir
cenfile = os.path.join(sample_dir(), 'legacyhalos-centrals-{}.fits'.format(lsdr))
candcenfile = os.path.join(sample_dir(), 'legacyhalos-candidate-centrals-{}.fits'.format(lsdr))
jackfile = os.path.join(sample_dir(), 'legacyhalos-jackknife-{}.fits'.format(lsdr))

### Read the matched Legacy Survey and redMaPPer catalogs.

In [11]:
def read_redmapper(rmversion='v6.3.1', index=None, satellites=False):
    """Read the parent redMaPPer cluster catalog and updated photometry.
    
    """
    if satellites:
        suffix1, suffix2 = '_members', '-members'
    else:
        suffix1, suffix2 = '', '-centrals'
    rmfile = os.path.join( os.getenv('REDMAPPER_DIR'), rmversion, 
                          'dr8_run_redmapper_{}_lgt5_catalog{}.fit'.format(rmversion, suffix1) )
    rmphotfile = os.path.join( os.getenv('REDMAPPER_DIR'), rmversion, 
                          'redmapper-{}-lgt5{}-sdssWISEphot-{}.fits'.format(rmversion, suffix2, sdssdr) )
    
    rm = Table(fitsio.read(rmfile, ext=1, upper=True, rows=index))
    rmphot = Table(fitsio.read(rmphotfile, ext=1, upper=True, rows=index))

    print('Read {} galaxies from {}'.format(len(rm), rmfile))
    print('Read {} galaxies from {}'.format(len(rmphot), rmphotfile))
    
    rm.rename_column('RA', 'RA_REDMAPPER')
    rm.rename_column('DEC', 'DEC_REDMAPPER')
    rmphot.rename_column('RA', 'RA_SDSS')
    rmphot.rename_column('DEC', 'DEC_SDSS')
    rmphot.rename_column('OBJID', 'SDSS_OBJID')

    assert(np.sum(rmphot['MEM_MATCH_ID'] - rm['MEM_MATCH_ID']) == 0)
    if satellites:
        assert(np.sum(rmphot['ID'] - rm['ID']) == 0)
        rm.remove_columns( ('ID', 'MEM_MATCH_ID') )
    else:
        rm.remove_column('MEM_MATCH_ID')
    rmout = hstack( (rmphot, rm) )
    del rmphot, rm

    # Add a central_id column
    #rmout.rename_column('MEM_MATCH_ID', 'CENTRAL_ID')
    #cid = ['{:07d}'.format(cid) for cid in rmout['MEM_MATCH_ID']]
    #rmout.add_column(Column(name='CENTRAL_ID', data=cid, dtype='U7'), index=0)
    
    return rmout

In [12]:
def read_legacysurvey(rmversion='v6.3.1', index=None, satellites=False, satid=None):
    """Read the matched Legacy Survey catalogs.
    
    Note that non-matching entries are populated with zeros / False.
    
    """
    if satellites:
        galtype = 'members'
    else:
        galtype = 'centrals'
       
    cols = ['RELEASE', 'BRICKID', 'BRICKNAME', 'OBJID', 'TYPE', 'RA', 'DEC',
            #'RA_IVAR', 'DEC_IVAR', 'DCHISQ', 
            'EBV', 'MASKBITS',
            #'FLUX_U', 'FLUX_I', 'FLUX_Y', 
            'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FLUX_W1', 'FLUX_W2', 'FLUX_W3', 'FLUX_W4', 
            #'FLUX_IVAR_U', 'FLUX_IVAR_I', 'FLUX_IVAR_Y',             
            'FLUX_IVAR_G', 'FLUX_IVAR_R', 'FLUX_IVAR_Z', 'FLUX_IVAR_W1', 'FLUX_IVAR_W2', 'FLUX_IVAR_W3', 'FLUX_IVAR_W4', 
            #'MW_TRANSMISSION_U', 'MW_TRANSMISSION_I', 'MW_TRANSMISSION_Y', 
            'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z', 
            'MW_TRANSMISSION_W1', 'MW_TRANSMISSION_W2', 'MW_TRANSMISSION_W3', 'MW_TRANSMISSION_W4', 
            #'NOBS_U', 'NOBS_I', 'NOBS_Y',
            'NOBS_G', 'NOBS_R', 'NOBS_Z', 'NOBS_W1', 'NOBS_W2', 'NOBS_W3', 'NOBS_W4',
            #'RCHISQ_U', 'RCHISQ_G', 'RCHISQ_R', 'RCHISQ_I', 'RCHISQ_Z', 'RCHISQ_Y',
            #'RCHISQ_W1', 'RCHISQ_W2', 'RCHISQ_W3', 'RCHISQ_W4', 
            #'FRACFLUX_U', 'FRACFLUX_I', 'FRACFLUX_Y', 
            'FRACFLUX_G', 'FRACFLUX_R', 'FRACFLUX_Z', 'FRACFLUX_W1', 'FRACFLUX_W2', 'FRACFLUX_W3', 'FRACFLUX_W4', 
            'FRACMASKED_G', 'FRACMASKED_R', 'FRACMASKED_Z', #'FRACMASKED_U', 'FRACMASKED_I', 'FRACMASKED_Y', 
            'FRACIN_G', 'FRACIN_R', 'FRACIN_Z', #'FRACIN_U', 'FRACIN_I', 'FRACIN_Y', 
            'ANYMASK_G', 'ANYMASK_R', 'ANYMASK_Z', #'ANYMASK_U', 'ANYMASK_I', 'ANYMASK_Y', 
            'ALLMASK_G', 'ALLMASK_R', 'ALLMASK_Z', 'WISEMASK_W1', 'WISEMASK_W2', #'ALLMASK_U', 'ALLMASK_I', 'ALLMASK_Y', 
            'PSFSIZE_G', 'PSFSIZE_R', 'PSFSIZE_Z', #'PSFSIZE_U', 'PSFSIZE_I', 'PSFSIZE_Y',
            'PSFDEPTH_G', 'PSFDEPTH_R', 'PSFDEPTH_Z', #'PSFDEPTH_U', 'PSFDEPTH_I', 'PSFDEPTH_Y', 
            'GALDEPTH_G', 'GALDEPTH_R', 'GALDEPTH_Z', #'GALDEPTH_U', 'GALDEPTH_I', 'GALDEPTH_Y', 
            'SERSIC', 'SERSIC_IVAR', 'SHAPE_R', 'SHAPE_R_IVAR', 'SHAPE_E1', 'SHAPE_E1_IVAR', 'SHAPE_E2', 'SHAPE_E2_IVAR']
    
    lsdr = 'dr9-north'
    lsfile = os.path.join( os.getenv('REDMAPPER_DIR'), rmversion, 
                          'legacysurvey-{}-{}-{}-lgt5.fits'.format(lsdr, galtype, rmversion) )
    dr9north = Table(fitsio.read(lsfile, ext=1, upper=True, rows=index, columns=cols))
    print('Read {} galaxies from {}'.format(len(dr9north), lsfile))

    lsdr = 'dr9-south'
    lsfile = os.path.join( os.getenv('REDMAPPER_DIR'), rmversion, 
                           'legacysurvey-{}-{}-{}-lgt5.fits'.format(lsdr, galtype, rmversion) )
    ls = Table(fitsio.read(lsfile, ext=1, upper=True, rows=index, columns=cols))
    print('Read {} galaxies from {}'.format(len(ls), lsfile))

    # Resolve north and south.
    print('Resolving catalogs at Dec>32.375')
    inorth = dr9north
    
    pdb.set_trace()
    
    # If both DR8-north and DR8-south, decide based on grz depth.
    both = (ls['RELEASE'] != 0) * (dr9north['RELEASE'] != 0)
    if np.sum(both) > 0:
        print('  Found {} galaxies with both north+south photometry.'.format(np.sum(both)))
        pdb.set_trace()
        usedr9north = ( (dr9north['PSFDEPTH_G'][both] > ls['PSFDEPTH_G'][both]) * 
                        (dr9north['PSFDEPTH_R'][both] > ls['PSFDEPTH_R'][both]) * 
                        (dr9north['PSFDEPTH_Z'][both] > ls['PSFDEPTH_Z'][both]) )
        if np.sum(usedr6) > 0:
            print('  Using deeper DR6 photometry for {}/{} galaxies.'.format(
                np.sum(usedr6), np.sum(both)))
            ls[both][usedr6] = dr6[both][usedr6]
            
    # If no DR7, use DR6.
    usedr6 = (ls['RELEASE'] == 0) * (dr6['RELEASE'] != 0)
    if np.sum(usedr6) > 0:
        print('  Using DR6 for {} galaxies without DR7 photometry.'.format(np.sum(usedr6)))
        ls[usedr6] = dr6[usedr6]

    # Next, we have to deal with the fact that the the redmapper catalog contains 
    # duplicates (via 'ID').  Consequently, the coordinate-matching code only 
    # matched to *one* of the members, but the rest of the code in this notebook 
    # needs all the entries populated (because although they have the same `ID`, they 
    # have different `MEM_MATCH_ID` values, i.e., they belong to different clusters).
    
    # For example, consider ID 23136319, which appears on rows 4161 and
    # 4632.  In the legacyhalos catalog only one entry is populated, e.g.,

    # RELEASE BRICKID BRICKNAME ... SHAPEEXP_E1_IVAR SHAPEEXP_E2 SHAPEEXP_E2_IVAR
    # int32   int32    bytes8  ...     float32        float32       float32
    # ------- ------- --------- ... ---------------- ----------- ----------------
    # 7000  498662  1402p305 ...              0.0         0.0              0.0
    #    0       0           ...              0.0         0.0              0.0

    # even though these are the same object.

    # The script below (written by Chun-Hao To) finds all the duplicates in the 
    # redmapper catalog (via 'ID'), find the entry in the legacyhalos catalog that 
    # is populated (e.g., with RELEASE != 0) and then copies over the data to
    # the entries that are empty.    
    
    if satellites:
        print('Processing duplicates in the satellites catalog.')
        t0 = time.time()
        
        redm_pd = pd.DataFrame({'ID': satid.byteswap().newbyteorder()})
        #redm_pd = pd.DataFrame.from_records(satid)
        redm_pd['index'] = pd.Series(np.arange(len(satid)), index=redm_pd.index)

        # Find duplicates
        duplicatedmask = redm_pd.duplicated(subset=['ID'], keep=False)
        redm_pd_duplicated = redm_pd[duplicatedmask]
        
        group = redm_pd_duplicated.groupby(['ID'])
        
        for name, grp in group:
            entry = None
            for index in grp['index']:
                temp = ls[index]
                if temp['RELEASE'] != 0:
                    entry = temp
            if entry is not None:
                for index in grp['index']:
                    ls[index] = entry
        print('    Time: {:.3f} min'.format((time.time() - t0)/60 ))
                
    miss = ls['RELEASE'] == 0
    print('A total of {}/{} galaxies ({:.2f}%) do not have DR9 photometry.'.format(
        np.sum(miss), len(ls), 100*np.sum(miss)/len(ls)))
    
    return ls

In [14]:
%time rmcenall = read_redmapper(rmversion=rmversion, index=index)

Read 396047 galaxies from /global/cfs/cdirs/desi/users/ioannis/redmapper/v6.3.1/dr8_run_redmapper_v6.3.1_lgt5_catalog.fit
Read 396047 galaxies from /global/cfs/cdirs/desi/users/ioannis/redmapper/v6.3.1/redmapper-v6.3.1-lgt5-centrals-sdssWISEphot-dr14.fits
CPU times: user 1.19 s, sys: 543 ms, total: 1.74 s
Wall time: 1.72 s


In [15]:
rmcenall

MEM_MATCH_ID,SDSS_OBJID,RUN,RERUN,CAMCOL,FIELD,RA_SDSS,DEC_SDSS,PETROMAGGIES [5],PETROMAGGIES_IVAR [5],MODELMAGGIES [5],MODELMAGGIES_IVAR [5],CMODELMAGGIES [5],CMODELMAGGIES_IVAR [5],WISE_NANOMAGGIES [2],WISE_NANOMAGGIES_IVAR [2],EXTINCTION [5],PETROR50 [5],RA_REDMAPPER,DEC_REDMAPPER,MODEL_MAG [5],MODEL_MAGERR [5],IMAG,IMAG_ERR,ZRED,ZRED_E,ZRED_CHISQ,BCG_SPEC_Z,Z_SPEC_INIT,Z_INIT,Z,LAMBDA_CHISQ,LAMBDA_CHISQ_E,LAMBDA_ZRED,LAMBDA_ZRED_E,R_LAMBDA,SCALEVAL,MASKFRAC,C_LAMBDA [4],C_LAMBDA_ERR [4],MAG_LAMBDA_ERR [5],CHISQ,Z_LAMBDA,Z_LAMBDA_E,Z_LAMBDA_NITER,EBV_MEAN,PHOTOID,LNLAMLIKE,LNBCGLIKE,LNLIKE,PZBINS [21],PZ [21],NCROSS,RMASK,RA_ORIG,DEC_ORIG,W,DLAMBDA_DZ,DLAMBDA_DZ2,DLAMBDAVAR_DZ,DLAMBDAVAR_DZ2,LAMBDA_CHISQ_C,LAMBDA_CHISQ_CE,NCENT,NCENT_GOOD,RA_CENT [5],DEC_CENT [5],ID_CENT [5],LAMBDA_CHISQ_CENT [5],ZLAMBDA_CENT [5],P_BCG [5],P_CEN [5],Q_CEN [5],P_FG [5],Q_MISS,P_SAT [5],P_C [5],BCG_ILUM,ILUM,Z_LAMBDA_RAW,Z_LAMBDA_E_RAW,LIM_EXPTIME,LIM_LIMMAG,LIM_LIMMAG_HARD
int32,int64,int64,int64,int64,int64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,int64,float32,float32,float32,float32,float32,int16,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,int16,int16,float64,float64,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
1,1237662340012638224,3927,301,2,85,239.58333543713738,27.23340954635452,95.80955 .. 4284.2686,0.00065498427 .. 1.6282245e-06,62.927708 .. 3331.9722,0.12114053 .. 0.0040006773,29.52413 .. 3215.089,0.53208333 .. 0.0043025617,39647.574 .. 42470.297,0.0004141982 .. 4.1722717e-05,0.18723309 .. 0.055785656,101.31456 .. 25.627058,239.5833290688836,27.233412917827273,17.76729 .. 13.619685,0.049571823 .. 0.0051517654,13.97901,0.002698147,0.09760145,0.005986461,17.529497,0.09080672,0.09080672,0.09760145,0.099599876,169.83784,4.1870255,167.90047,3.827136,1.1117494,1.0113665,0.012301013,1.9110068 .. 0.3405583,0.004784675 .. 0.00015905016,0.004719596 .. 0.0,17.16032,0.09483067,0.004581776,1,0.015627198,39270301200850016,417.53326,3.9817564,421.515,0.06894211 .. 0.120719224,6.6654225e-06 .. 8.297843e-06,1,1.6676241,239.5833290688836,27.233412917827273,1.0926445,22.14292,-1019.208,431.90344,85493.54,169.36815,1.2413765,5,2,239.5833290688836 .. 239.5596341733162,27.233412917827273 .. 27.272102672585504,25404293 .. 25404056,169.83784 .. 165.79527,0.096497506 .. 0.0,1.0 .. 1.0,0.87483835 .. 5.368429e-07,0.7527664 .. 4.6193367e-07,0.00018875106 .. 0.0025028393,-4.7970977,0.124972895 .. 0.9974966,0.84362197 .. 3.3104739e-06,7.4597607,115.149506,0.096497506,0.0015025047,122.96634,21.177217,21.2911
2,1237651715872325879,1453,301,5,57,250.082552977366,46.71147489348557,19.337973 .. 200.16772,0.07398379 .. 0.00086578116,10.73619 .. 460.24417,0.48645884 .. 0.014912097,31.426146 .. 274.85587,0.07443199 .. 0.040102884,7234.426 .. 8665.855,0.0030079759 .. 0.00028875555,0.10203367 .. 0.03040069,10.49419 .. 2.44081,250.08254838650885,46.71153135360437,19.793066 .. 15.814708,0.14495553 .. 0.019318104,16.290333,0.0075439033,0.26355553,0.007049759,16.153416,-1.0,-1.0,0.23313455,0.2333275,199.53928,5.2998266,173.62108,5.049672,1.1481687,0.99517053,0.0,1.8007203 .. 0.3218498,0.11659203 .. 0.0017549456,0.1152135 .. 0.0,22.234745,0.23276423,0.0061555807,1,0.026837764,14530301500570247,382.43182,1.1580691,383.5899,0.19623154 .. 0.26929685,3.38395e-07 .. 5.934771e-06,1,1.7222531,250.08647635515513,46.71226813734512,1.2596524,7.3045945,-56.40334,228.87538,22720.906,199.5935,0.4635124,5,2,250.08254838650885 .. 250.10757652178705,46.71153135360437 .. 46.70946459390802,3352952 .. 3352965,199.53928 .. 210.08046,0.23142491 .. 0.0,1.0 .. 1.0,0.98567283 .. 3.2535431e-06,0.63352615 .

In [13]:
%time lscenall = read_legacysurvey(rmversion=rmversion, index=index)

Read 396047 galaxies from /global/cfs/cdirs/desi/users/ioannis/redmapper/v6.3.1/legacysurvey-dr9-north-centrals-v6.3.1-lgt5.fits
Read 396047 galaxies from /global/cfs/cdirs/desi/users/ioannis/redmapper/v6.3.1/legacysurvey-dr9-south-centrals-v6.3.1-lgt5.fits
Resolving catalogs at Dec>32.375
> <ipython-input-12-10f0bbe180ec>(56)read_legacysurvey()
     54 
     55     # If both DR8-north and DR8-south, decide based on grz depth.
---> 56     both = (ls['RELEASE'] != 0) * (dr9north['RELEASE'] != 0)
     57     if np.sum(both) > 0:
     58         print('  Found {} galaxies with both north+south photometry.'.format(np.sum(both)))



ipdb>  q


BdbQuit: 

In [ ]:
index = np.arange(1000)
#%time lssatall = read_legacysurvey(rmversion=rmversion, satellites=True, satid=rmsatall['ID'].data[index], index=index) # rmsat=rmsatall)
%time lssatall = read_legacysurvey(rmversion=rmversion)

In [ ]:
#ii = [[4161, 4632]]
#lssatall[[4161, 4632]]
#rmsatall[ii]

### Centrals
Require a match with redMaPPer and non-zero depth in all three bands.

In [ ]:
def select_legacysurvey(lscat, rmcat):
    good = np.where(
        (lscat['GALDEPTH_G'] > 0) * 
        (lscat['GALDEPTH_R'] > 0) * 
        (lscat['GALDEPTH_Z'] > 0) * 
        (lscat['NOBS_G'] > 1) * 
        (lscat['NOBS_R'] > 1) * 
        (lscat['NOBS_Z'] > 1) *
        (np.sum(rmcat['MODELMAGGIES'] == 0, axis=1) != 5) )[0] # missing SDSS photometry
    return good

In [ ]:
lscenall = read_legacysurvey(rmversion=rmversion, index=index)
assert(len(rmcenall) == len(lscenall))

In [ ]:
cenmatched = select_legacysurvey(lscenall, rmcenall)
print('Identified {} / {} ({:.2f}%) centrals with grz photometry (nobs>1) and a match to redMaPPer.'.format(
    len(cenmatched), len(lscenall), 100*len(cenmatched)/len(lscenall)))
lscen = lscenall[cenmatched]
lscen

In [ ]:
rmcen = rmcenall[cenmatched]
rmcen

In [ ]:
#plt.scatter( lscen['RA'], (lscen['RA'] - rmcen['RA_SDSS']) * 3600, s=1)
#plt.scatter( lscen['DEC'], (lscen['DEC'] - rmcen['DEC_SDSS']) * 3600, s=1)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(rmcenall['RA_SDSS'], rmcenall['DEC_SDSS'], s=1, label='redMaPPer/v6.3.1')
ax.scatter(rmcen['RA_SDSS'], rmcen['DEC_SDSS'], s=1, alpha=0.1, 
           marker='.', label='DR6/DR7 Matched')
ax.set_xlabel('RA')
ax.set_ylabel('Dec')
ax.set_ylim(-20, 80)
ax.invert_xaxis()
lgnd = ax.legend(loc='upper left', frameon=False, fontsize=10, ncol=2)
for ll in lgnd.legendHandles:
    ll._sizes = [30]

### Unpack the candidate central galaxies from the satellites / members catalog.
We are not analyzing the full set of satellites.

In [ ]:
%time rmsatall = read_redmapper(rmversion=rmversion, satellites=True, index=index)

In [ ]:
if index is None:
    satid = rmsatall['ID'].data
else:
    satid = rmsatall['ID'].data[index]
%time lssatall = read_legacysurvey(rmversion=rmversion, satellites=True, satid=satid, index=index)
assert(len(rmsatall) == len(lssatall))

In [ ]:
def get_central_candidates(cen, sat, ls):
    """Create a hash table connecting, for each cluster, all the candidate 
    centrals' ID numbers to an index in the satellites catalog.  The clever 
    algorithm used here is by Chun-Hao To (Stanford).
    
    """    
    ncen, ncand = cen['ID_CENT'].shape

    #offset = sat['ID'].min()
    #g_index = dok_matrix( (np.max(sat['ID']) - offset + 1, 1), dtype=np.int )
    #g_index[sat['ID'] - offset] = np.array( range( len(sat) ) )[:, np.newaxis]
    
    # Create a DataFrame for the catalog of centrals.
    cen_temp = [cen['ID_CENT'][:, ii] for ii in range(ncand)]
    cen_temp.append(cen['MEM_MATCH_ID'])
    columns = ['ID_CENT_{}'.format(ii) for ii in range(ncand)]
    columns.append('MEM_MATCH_ID_CEN')
               
    cen_pd = pd.DataFrame.from_records(np.array(cen_temp).T, columns=columns)
    del cen_temp, columns

    # Create DataFrame for the satellites / members.
    sat_pd = pd.DataFrame.from_records(sat[['ID', 'MEM_MATCH_ID']].as_array())
    sat_pd['index'] = pd.Series(np.arange(len(sat)), index=sat_pd.index)

    # Create the mapping between them
    cengalindex = np.zeros_like(cen['ID_CENT'])
    pcen = np.zeros( len(sat) ).astype('f4')
    primary_central = np.zeros( len(sat) ).astype(bool)
    
    for ii in range(ncand):
        # Old algorithm which doesn't deal with duplicates correctly.
        #index = np.where( cen['ID_CENT'][:, ii] - offset >= 0 )[0]
        #cengalindex[index, ii] = g_index[cen['ID_CENT'][index, ii] - offset]
        merged = pd.merge(cen_pd, sat_pd, left_on=['ID_CENT_{}'.format(ii), 'MEM_MATCH_ID_CEN'], 
                          right_on=['ID', 'MEM_MATCH_ID'], suffixes=('_original','_matched'))
        cengalindex[:, ii] = merged['index']
        pcen[cengalindex[:, ii]] = cen['P_CEN'][:, ii]
        if ii == 0:
            primary_central[cengalindex[:, ii]] = True
        
    cengalindex = cengalindex.flatten()
        
    candcen = sat[cengalindex]
    candcen.add_column(Column(name='P_CEN', data=pcen[cengalindex]), index=1)
    candcen.add_column(Column(name='PRIMARY_CENTRAL', data=primary_central[cengalindex]), index=2)

    return candcen, ls[cengalindex]

In [ ]:
%time rmcandcenall, lscandcenall = get_central_candidates(rmcenall, rmsatall, lssatall)

In [ ]:
candcenmatched = select_legacysurvey(lscandcenall, rmcandcenall)
print('Identified {} / {} ({:.2f}%) candidate centrals with grz photometry and a match to redMaPPer.'.format(
    len(candcenmatched), len(lscandcenall), 100*len(candcenmatched)/len(lscandcenall)))
lscandcen = lscandcenall[candcenmatched]
#lscandcen

In [ ]:
rmcandcen = rmcandcenall[candcenmatched]
rmcandcen

### Additional sample cuts

Exclude the sources with the shallowest DR6/DR7 *grz* photometry based on the estimated point-source depth.

In [ ]:
band = ['G', 'R', 'Z']
targdepth = [24.0, 23.4, 22.5] # target 5-sigma depth
meddepth, P10depth = np.zeros((3)), np.zeros((3))

In [ ]:
#dd = 22.5-2.5*np.log10(5/np.sqrt(lscen['PSFDEPTH_Z']))
#nobs = lscen['NOBS_G']
#_ = plt.hist(dd, bins=100)
#print(dd.min(), np.sum(dd < 22), np.sum(nobs <= 1))

In [ ]:
color = ('blue', 'green', 'red')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4), sharey=True)
for ii, (tt, bb, col) in enumerate( zip( targdepth, band, color ) ):
    cendepth = 22.5 - 2.5 * np.log10( 5 / np.sqrt(lscen['PSFDEPTH_{}'.format(bb)]) )
    candcendepth = 22.5 - 2.5 * np.log10( 5 / np.sqrt(lscandcen['PSFDEPTH_{}'.format(bb)]) )
    
    meddepth[ii] = np.percentile(cendepth, [50])
    P10depth[ii] = np.percentile(cendepth, [10])
    print('{} depth: P10: {:.3f}, median = {:.3f}, target = {:.3f}'.format(
        bb.lower(), P10depth[ii], meddepth[ii], tt))
    
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        # centrals
        nn, bins, patches = ax1.hist(cendepth, bins=100, histtype='step', cumulative=True,
                                     label=bb.lower(), normed=True, color=col, lw=2)
        patches[0].set_xy(patches[0].get_xy()[:-1]) # delete the last point
        # candidate centrals
        nn, bins, patches = ax2.hist(candcendepth, bins=100, histtype='step', cumulative=True,
                                     label=bb.lower(), normed=True, color=col, lw=2)
        patches[0].set_xy(patches[0].get_xy()[:-1]) # delete the last point
    
    #ax.axvline(x=tt, ls='--', color=col, lw=2, alpha=1.0)
    #ax.axvline(x=meddepth[ii], ls='-', color=col, lw=1, alpha=0.9)
    for ax in (ax1, ax2):
        ax.axvline(x=P10depth[ii], ls='-', color=col, lw=1, alpha=0.9)

ax1.legend(loc='upper left')
ax1.set_ylabel('Fraction of Sample')
ax1.set_title('Primary Centrals')
ax2.set_title('Candidate Centrals')
for ax in (ax1, ax2):
    ax.set_xlabel('Imaging Depth (5$\sigma$, AB mag)')
    ax.set_xlim(21, 26)
    
fig.subplots_adjust(hspace=0.01)

In [ ]:
depthcut = (23.5, 23.0, 22.0)
cendepthcut = np.ones(len(lscen)).astype(bool)
candcendepthcut = np.ones(len(lscandcen)).astype(bool)
for ii, bb in enumerate(['G', 'R', 'Z']):
    cendepth = 22.5 - 2.5 * np.log10( 5 / np.sqrt(lscen['PSFDEPTH_{}'.format(bb)]) )
    candcendepth = 22.5 - 2.5 * np.log10( 5 / np.sqrt(lscandcen['PSFDEPTH_{}'.format(bb)]) )
    cendepthcut *= cendepth > depthcut[ii]
    candcendepthcut *= candcendepth > depthcut[ii]
    #cendepthcut *= cendepth > p10depth[ii]
    #satdepthcut *= satdepth > p10depth[ii]
print('{} / {} ({:.2f}%) centrals pass the depth cuts in all three bands.'.format(
    np.sum(cendepthcut), len(lscen), 100*np.sum(cendepthcut)/len(lscen)))
print('{} / {} ({:.2f}%) candidate centrals pass the depth cuts in all three bands.'.format(
    np.sum(candcendepthcut), len(lscandcen), 100*np.sum(candcendepthcut)/len(lscandcen)))

### Cut and join the redMaPPer (central & satellite) and LS catalogs

In [ ]:
rmcendeep = rmcen[cendepthcut]
lscendeep = lscen[cendepthcut]
cen = hstack( (rmcendeep, lscendeep) )

In [ ]:
candcen = hstack( (rmcandcen[candcendepthcut], lscandcen[candcendepthcut]) )
#del rmcandcen, lscandcen

In [ ]:
def _normhist(xx, ax, label=None, alpha=1.0, lw=2, bins=100):
    _, _, _ = ax.hist(xx, weights=np.ones_like(xx) / float(len(xx)), bins=bins, 
                      histtype='step', label=label, lw=lw, alpha=alpha)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4), sharey=True)

_normhist(rmcenall['Z'], ax1, label='All Centrals', lw=3)
_normhist(rmcen['Z'], ax1, label='Matching Centrals', alpha=0.8)
_normhist(rmcendeep['Z'], ax1, label='After Depth Cuts', alpha=0.8)
ax1.set_xlabel('Redshift')
ax1.set_ylabel('Fraction of Centrals')
ax1.legend(loc='upper left')

_normhist(np.log10(rmcenall['LAMBDA_CHISQ']), ax2, lw=3)
_normhist(np.log10(rmcen['LAMBDA_CHISQ']), ax2, alpha=0.8)
_normhist(np.log10(rmcendeep['LAMBDA_CHISQ']), ax2, alpha=0.8)
ax2.set_xlabel('$\log_{10}$ (Cluster Richness $\lambda$)')
#ax2.set_ylabel('Fraction of Galaxies')
fig.subplots_adjust(wspace=0.03)

### Get the total area subtended by the final sample.

In [ ]:
def get_area(nside=256, qaplot=True):
    """Get the unique area of the sample."""
    
    areaperpix = hp.nside2pixarea(nside, degrees=True)
    samplepix = radec2pix(nside, cen['RA'].data, cen['DEC'].data)
    print('Subdividing the sample into nside={} healpixels with area={:.4f} deg2 per pixel.'.format(
        nside, areaperpix))

    outpixmap = []
    for dr, release in zip( ('dr6.0', 'dr7.1'), (6000, 7000) ):
        # Read the pixel weight map which quantifies the imaging footprint
        pixfile = os.path.join( legacyhalos.io.sample_dir(), 'pixweight-{}-0.22.0.fits'.format(dr) )
        pixmap = Table(fitsio.read(pixfile))
        pixmap['DR'] = dr.upper()
    
        these = cen['RELEASE'].data == release
        thesepix = np.unique(samplepix[these])
    
        # Only keep non-empty healpixels.
        keep = ( (pixmap['FRACAREA'][thesepix] > 0) * 
                (pixmap['PSFDEPTH_G'][thesepix] > 0) * # p10depth[0]) * 
                (pixmap['PSFDEPTH_R'][thesepix] > 0) * # p10depth[1]) * 
                (pixmap['PSFDEPTH_Z'][thesepix] > 0)   # p10depth[2]) 
               )
        outpixmap.append(pixmap[thesepix][keep])
    outpixmap = vstack(outpixmap)
    
    if False:
        print('Clamping FRACAREA at unity!')
        toobig = outpixmap['FRACAREA'] > 1
        if np.sum(toobig) > 0:
            outpixmap['FRACAREA'][toobig] = 1.0

    # Don't double-count area, where DR6 and DR7 overlap.
    _, keep = np.unique(outpixmap['HPXPIXEL'], return_index=True)
    dup = np.delete( np.arange(len(outpixmap)), keep )
    
    # Code to double-check for duplicates and to ensure every object 
    # has been assigned a healpixel.
    # for pp in outpixmap['HPXPIXEL'][keep]:
    #     if np.sum( pp == outpixmap['HPXPIXEL'][keep] ) > 1:
    #         print('Duplicate!')
    #         import pdb ; pdb.set_trace()
    #     if np.sum( pp == samplepix ) == 0:
    #         print('Missing healpixel!')
    #         import pdb ; pdb.set_trace()
    
    area = np.sum(outpixmap['FRACAREA'][keep]) * areaperpix
    duparea = np.sum(outpixmap['FRACAREA'][dup]) * areaperpix

    if qaplot:
        uu = np.in1d(samplepix, outpixmap['HPXPIXEL'][keep])
        dd = np.in1d(samplepix, outpixmap['HPXPIXEL'][dup])
        fig, ax = plt.subplots()
        ax.scatter(cen['RA'][uu], cen['DEC'][uu], s=1, marker='s',
                   label=r'Unique: {:.1f} deg$^{{2}}$'.format(area))
        ax.scatter(cen['RA'][dd], cen['DEC'][dd], s=1, marker='s',
                   label=r'Overlapping: {:.1f} deg$^{{2}}$'.format(duparea))
        ax.set_xlim(0, 360)
        ax.set_ylim(-15, 80)
        #ax.legend(loc='upper right', fontsize=12, frameon=False)
        ax.invert_xaxis()
        lgnd = ax.legend(loc='upper left', frameon=False, fontsize=10, ncol=2)
        for ll in lgnd.legendHandles:
            ll._sizes = [30]        
        
    return area, duparea, outpixmap[keep]

In [ ]:
area, duparea, pixmap = get_area()
print('Unique area = {:.3f} deg2\nOverlapping area = {:.3f} deg2'.format(area, duparea))

In [ ]:
len(pixmap), len(np.unique(pixmap['HPXPIXEL']))

### Create jackknife samples

In [ ]:
def jackknife_samples(pixmap, nside_pixmap=256, nside_jack=4):
    """Split the sample into ~equal area chunks and write out a table.
    
    """
    from astropy.io import fits
    
    area_jack = hp.nside2pixarea(nside_jack, degrees=True)
    area_pixmap = hp.nside2pixarea(nside_pixmap, degrees=True)
    print('Jackknife nside = {} with area = {:.3f} deg2'.format(nside_jack, area_jack))
    
    pix_jack = radec2pix(nside_jack, cen['RA'].data, cen['DEC'].data)
    pix_pixmap = radec2pix(nside_pixmap, cen['RA'].data, cen['DEC'].data)
    
    upix_jack = np.unique(pix_jack)
    upix_jack = upix_jack[np.argsort(upix_jack)]
    npix = len(upix_jack)
    
    ra_jack, dec_jack = pix2radec(nside_jack, upix_jack)
    
    out = Table()
    out['HPXPIXEL'] = upix_jack
    out['RA'] = ra_jack
    out['DEC'] = dec_jack
    out['AREA'] = np.zeros(npix).astype('f4')
    out['NCEN'] = np.zeros(npix).astype('int')
    
    for ii, pp in enumerate(upix_jack):
        these = np.where( pp == pix_jack )[0]
        indx = np.where( np.in1d( pixmap['HPXPIXEL'].data, pix_pixmap[these] ) )[0]
        uindx = np.unique(indx)
        #print(pp, len(indx), len(uindx))

        out['AREA'][ii] = np.sum(pixmap['FRACAREA'][indx].data) * area_pixmap
        out['NCEN'][ii] = len(these)
        
        #if ii == 4:
        #    rbig, dbig = pix2radec(nside_jack, pp)
        #    rsmall, dsmall = pix2radec(nside_pixmap, pixmap['HPXPIXEL'][indx].data)
        #    rgal, dgal = sample['RA'][these], sample['DEC'][these]
        #    plt.scatter(rgal, dgal, s=3, marker='o', color='green')
        #    plt.scatter(rsmall, dsmall, s=3, marker='s', color='blue')
        #    plt.scatter(rbig, dbig, s=75, marker='x', color='k')
        #    plt.show()
        #    import pdb ; pdb.set_trace() 
        
    print('Writing {}'.format(jackfile))
    hx = fits.HDUList()
    hdu = fits.convenience.table_to_hdu(out)
    hdu.header['EXTNAME'] = 'JACKKNIFE'
    hdu.header['NSIDE'] = nside_jack
    hx.append(hdu)
    hx.writeto(jackfile, overwrite=True)

    return out

In [ ]:
nside_jack = 4
jack = jackknife_samples(pixmap, nside_jack=nside_jack)
njack = len(jack)
jack

In [ ]:
print('Check: total area = {:.3f}, total number of galaxies = {}'.format(
    np.sum(jack['AREA']), np.sum(jack['NCEN'])))
print('Mean / median area per pixel = {:.3f} / {:.3f} deg2'.format(
    np.mean(jack['AREA']), np.median(jack['AREA'])))
print('Mean / median number of centrals per pixel = {:.0f} / {:.0f}'.format(
    np.mean(jack['NCEN']), np.median(jack['NCEN'])))
_ = plt.hist(jack['AREA'])

Visualize the jackknife samples.

In [ ]:
jackpix = legacyhalos.misc.radec2pix(nside_jack, cen['RA'].data, cen['DEC'].data)
jack_ra, jack_dec = legacyhalos.misc.pix2radec(nside_jack, jack['HPXPIXEL'])
fig, ax = plt.subplots()
for ii in range(njack):
    indx = np.where( jack['HPXPIXEL'][ii] == jackpix )[0]
    ax.scatter(cen['RA'][indx], cen['DEC'][indx], s=1)
    ax.text(jack_ra[ii], jack_dec[ii], '{:02d}'.format(ii), 
            va='center', ha='center')
    #ax.text(jack_ra[ii], jack_dec[ii], '{:02d}'.format(jack['HPXPIXEL'][ii]))    
ax.set_xlabel('RA')
ax.set_ylabel('Dec')

### Write out the final samples.

In [ ]:
print('Writing {}'.format(cenfile))
cen.write(cenfile, overwrite=True)

In [ ]:
print('Writing {}'.format(candcenfile))
candcen.write(candcenfile, overwrite=True)

In [ ]:
#bb = lscandcenall[rmcandcenall['ID'] == 25404292]
#bb[['GALDEPTH_G', 'GALDEPTH_R', 'GALDEPTH_Z', 'NOBS_G', 'NOBS_R', 'NOBS_Z']]